In [47]:
import pandas as pd
import joblib

In [48]:
print("="*60)
print("LOADING MODEL AND ENCODERS")
print("="*60)

# Load the trained model, scaler, and encoders
model = joblib.load("train_models/lung_cancer_model.pkl")
scaler = joblib.load("train_models/scaler.pkl")
encoders = joblib.load("train_models/encoders.pkl")

print("✅ Model loaded successfully!")
print("✅ Scaler loaded successfully!")
print("✅ Encoders loaded successfully!")

LOADING MODEL AND ENCODERS
✅ Model loaded successfully!
✅ Scaler loaded successfully!
✅ Encoders loaded successfully!


In [ ]:
# Print encoding mappings for reference
print("\n" + "="*60)
print("ENCODING REFERENCE")
print("="*60)
for col, encoder in encoders.items():
    if col != 'target':  # Skip target
        mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
        print(f"{col}: {mapping}")




ENCODING REFERENCE
gender: {'F': np.int64(0), 'M': np.int64(1)}
smoking: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
yellow_fingers: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
anxiety: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
peer_pressure: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
chronic_disease: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
fatigue: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
allergy: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
wheezing: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
alcohol_consuming: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
coughing: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
shortness_of_breath: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
swallowing_difficulty: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}
chest_pain: {np.int64(1): np.int64(0), np.int64(2): np.int64(1)}

MAKING PREDICTIONS


In [56]:
print("\n" + "="*60)
print("MAKING PREDICTIONS")
print("="*60)
# Helper function to make predictions
def predict_cancer(patient_data):
    """
    Make a prediction for a patient
    
    Args:
        patient_data: dict with raw values (e.g., 'M', 'F', 1, 2, actual age)
    
    Returns:
        prediction and confidence
    """
    # Create DataFrame
    patient_df = pd.DataFrame([patient_data])
    
    # Encode only categorical columns using saved encoders
    for col in patient_df.columns:
        if col in encoders:  # Only encode if it's a categorical column
            patient_df[col] = encoders[col].transform(patient_df[col])
        # else: keep numeric values (like age) as-is
    
    # Scale
    patient_scaled = scaler.transform(patient_df)
    
    # Predict
    prediction = model.predict(patient_scaled)
    prediction_proba = model.predict_proba(patient_scaled)
    
    result = "Lung Cancer" if prediction[0] == 1 else "No Lung Cancer"
    confidence = round(prediction_proba[0][prediction[0]] * 100, 2)
    
    return result, confidence



MAKING PREDICTIONS


In [51]:
print("\n🩺 TEST 1: High-Risk Patient")
print("-" * 60)
high_risk = {
    'gender': 'M',
    'age': 69,
    'smoking': 1,
    'yellow_fingers': 2,
    'anxiety': 2,
    'peer_pressure': 1,
    'chronic_disease': 1,
    'fatigue': 2,
    'allergy': 1,
    'wheezing': 2,
    'alcohol_consuming': 2,
    'coughing': 2,
    'shortness_of_breath': 2,
    'swallowing_difficulty': 2,
    'chest_pain': 2
}

result, confidence = predict_cancer(high_risk)
print("Patient Data:", high_risk)
print(f"🔮 Prediction: {result}")
print(f"🧩 Confidence: {confidence}%")


🩺 TEST 1: High-Risk Patient
------------------------------------------------------------
Patient Data: {'gender': 'M', 'age': 69, 'smoking': 1, 'yellow_fingers': 2, 'anxiety': 2, 'peer_pressure': 1, 'chronic_disease': 1, 'fatigue': 2, 'allergy': 1, 'wheezing': 2, 'alcohol_consuming': 2, 'coughing': 2, 'shortness_of_breath': 2, 'swallowing_difficulty': 2, 'chest_pain': 2}
🔮 Prediction: Lung Cancer
🧩 Confidence: 99.39%


In [52]:
# Example 2: Low-risk patient (from dataset: F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO)
print("\n🩺 TEST 2: Low-Risk Patient")
print("-" * 60)
low_risk = {
    'gender': 'F',
    'age': 59,
    'smoking': 1,
    'yellow_fingers': 1,
    'anxiety': 1,
    'peer_pressure': 2,
    'chronic_disease': 1,
    'fatigue': 2,
    'allergy': 1,
    'wheezing': 2,
    'alcohol_consuming': 1,
    'coughing': 2,
    'shortness_of_breath': 2,
    'swallowing_difficulty': 1,
    'chest_pain': 2
}

result, confidence = predict_cancer(low_risk)
print("Patient Data:", low_risk)
print(f"🔮 Prediction: {result}")
print(f"🧩 Confidence: {confidence}%")


🩺 TEST 2: Low-Risk Patient
------------------------------------------------------------
Patient Data: {'gender': 'F', 'age': 59, 'smoking': 1, 'yellow_fingers': 1, 'anxiety': 1, 'peer_pressure': 2, 'chronic_disease': 1, 'fatigue': 2, 'allergy': 1, 'wheezing': 2, 'alcohol_consuming': 1, 'coughing': 2, 'shortness_of_breath': 2, 'swallowing_difficulty': 1, 'chest_pain': 2}
🔮 Prediction: Lung Cancer
🧩 Confidence: 83.21%


In [53]:
# Example 3: Another NO case (F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO)
print("\n🩺 TEST 3: Another Low-Risk Patient")
print("-" * 60)
another_no = {
    'gender': 'F',
    'age': 63,
    'smoking': 1,
    'yellow_fingers': 2,
    'anxiety': 1,
    'peer_pressure': 1,
    'chronic_disease': 1,
    'fatigue': 1,
    'allergy': 1,
    'wheezing': 2,
    'alcohol_consuming': 1,
    'coughing': 2,
    'shortness_of_breath': 2,
    'swallowing_difficulty': 1,
    'chest_pain': 1
}

result, confidence = predict_cancer(another_no)
print("Patient Data:", another_no)
print(f"🔮 Prediction: {result}")
print(f"🧩 Confidence: {confidence}%")


🩺 TEST 3: Another Low-Risk Patient
------------------------------------------------------------
Patient Data: {'gender': 'F', 'age': 63, 'smoking': 1, 'yellow_fingers': 2, 'anxiety': 1, 'peer_pressure': 1, 'chronic_disease': 1, 'fatigue': 1, 'allergy': 1, 'wheezing': 2, 'alcohol_consuming': 1, 'coughing': 2, 'shortness_of_breath': 2, 'swallowing_difficulty': 1, 'chest_pain': 1}
🔮 Prediction: No Lung Cancer
🧩 Confidence: 86.33%


In [54]:
# Example 4: Custom patient - Very healthy young person
print("\n🩺 TEST 4: Very Healthy Young Person")
print("-" * 60)
healthy = {
    'gender': 'F',
    'age': 30,
    'smoking': 1,
    'yellow_fingers': 1,
    'anxiety': 1,
    'peer_pressure': 1,
    'chronic_disease': 1,
    'fatigue': 1,
    'allergy': 1,
    'wheezing': 1,
    'alcohol_consuming': 1,
    'coughing': 1,
    'shortness_of_breath': 1,
    'swallowing_difficulty': 1,
    'chest_pain': 1
}

result, confidence = predict_cancer(healthy)
# result, confidence = predict_cancer(another_no)
print("Patient Data:", healthy)
print(f"🔮 Prediction: {result}")
print(f"🧩 Confidence: {confidence}%")


print("\n" + "="*60)
print("PREDICTION COMPLETED")
print("="*60)
print("\n💡 To predict for your own patient:")
print("   Use the predict_cancer() function with raw values")
print("   Example: predict_cancer({'gender': 'M', 'age': 55, ...})")


🩺 TEST 4: Very Healthy Young Person
------------------------------------------------------------
Patient Data: {'gender': 'F', 'age': 30, 'smoking': 1, 'yellow_fingers': 1, 'anxiety': 1, 'peer_pressure': 1, 'chronic_disease': 1, 'fatigue': 1, 'allergy': 1, 'wheezing': 1, 'alcohol_consuming': 1, 'coughing': 1, 'shortness_of_breath': 1, 'swallowing_difficulty': 1, 'chest_pain': 1}
🔮 Prediction: No Lung Cancer
🧩 Confidence: 99.68%

PREDICTION COMPLETED

💡 To predict for your own patient:
   Use the predict_cancer() function with raw values
   Example: predict_cancer({'gender': 'M', 'age': 55, ...})
